# buffer

In [12]:
import numpy as np
from collections import deque
import random

In [13]:
class BasicBuffer_a:
    
    def __init__(self, size, obs_dim, act_dim):
        self.obs1_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.obs2_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size, act_dim], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros([size], dtype=np.float32)
        self.ptr, self.size, self.max_size = 0, 0, size

    def push(self, obs, act, rew, next_obs, done):
        self.obs1_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = np.asarray([rew])
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size)
        temp_dict= dict(s=self.obs1_buf[idxs],
                    s2=self.obs2_buf[idxs],
                    a=self.acts_buf[idxs],
                    r=self.rews_buf[idxs],
                    d=self.done_buf[idxs])
        return (temp_dict['s'],temp_dict['a'],temp_dict['r'].reshape(-1,1),temp_dict['s2'],temp_dict['d'])

# Critic and agent

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate

In [15]:
def Critic_gen(state_size, action_size,hidden_layers):

    input_x = Input(shape=state_size)
    input_a = Input(shape=action_size)
    x = input_x
    for i,j in enumerate(hidden_layers[:-1]):
        if i==1:
            x = concatenate([x,input_a],axis=-1)
        x = Dense(j,activation='relu')(x)
    x = Dense(hidden_layers[-1])(x)
    
    return tf.keras.Model([input_x,input_a],x)

def Actor_gen(state_size,action_size,hidden_layers,action_mult=1):
    input_x = Input(shape=state_size)
    x = input_x
    for i in hidden_layers:
        x = Dense(i,activation='relu')(x)
    x = Dense(action_size,activation='tanh')(x)
    x = tf.math.multiply(x,action_mult)
    return tf.keras.Model(input_x,x)

# DDPG

In [16]:
import matplotlib.pyplot as plt
from datetime import datetime
from sys import exit

In [17]:
class DDPGAgent:
    
    def __init__(self, env, gamma, tau, buffer_maxlen, critic_learning_rate, actor_learning_rate):
        
        self.env = env
        self.obs_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]
        self.action_max = env.action_space.high[0]
        # self.action_max = 1
        
        # hyperparameters
        self.env = env
        self.gamma = gamma
        self.tau = tau
        
        #Network layers
        actor_layer = [512,200,128]
        critic_layer = [1024,512,300,1]


        # Main network outputs
        self.mu = Actor_gen((3),(1),actor_layer,self.action_max)
        self.q_mu = Critic_gen((3),(1),critic_layer)

        # Target networks
        self.mu_target = Actor_gen((3),(1),actor_layer,self.action_max)
        self.q_mu_target = Critic_gen((3),(1),critic_layer)
      
        # Copying weights in,
        self.mu_target.set_weights(self.mu.get_weights())
        self.q_mu_target.set_weights(self.q_mu.get_weights())
    
        # optimizers
        self.mu_optimizer = tf.keras.optimizers.Adam(learning_rate=critic_learning_rate)
        self.q_mu_optimizer = tf.keras.optimizers.Adam(learning_rate=actor_learning_rate)
  
        self.replay_buffer = BasicBuffer_a(size = buffer_maxlen, obs_dim = self.obs_dim, act_dim = self.action_dim)
        
        self.q_losses = []
        
        self.mu_losses = []
        
    def get_action(self, s, noise_scale):
        a =  self.mu.predict(s.reshape(1,-1))[0]
        a += noise_scale * np.random.randn(self.action_dim)
        return np.clip(a, -self.action_max, self.action_max)

    def update(self, batch_size):
        
        
        X,A,R,X2,D = self.replay_buffer.sample(batch_size)
        X = np.asarray(X,dtype=np.float32)
        A = np.asarray(A,dtype=np.float32)
        R = np.asarray(R,dtype=np.float32)
        X2 = np.asarray(X2,dtype=np.float32)
        
        
        Xten=tf.convert_to_tensor(X)
        

        # Updating Critic
        with tf.GradientTape() as tape:
            A2 =  self.mu_target(X2)
            q_target = R + self.gamma  * self.q_mu_target([X2,A2])
            qvals = self.q_mu([X,A]) 
            q_loss = tf.reduce_mean((qvals - q_target)**2)
            grads_q = tape.gradient(q_loss,self.q_mu.trainable_variables)
        self.q_mu_optimizer.apply_gradients(zip(grads_q, self.q_mu.trainable_variables))
        self.q_losses.append(q_loss)


        #Updating Actor
        with tf.GradientTape() as tape2:
            A_mu =  self.mu(X)
            Q_mu = self.q_mu([X,A_mu])
            mu_loss =  -tf.reduce_mean(Q_mu)
            grads_mu = tape2.gradient(mu_loss,self.mu.trainable_variables)
        self.mu_losses.append(mu_loss)
        self.mu_optimizer.apply_gradients(zip(grads_mu, self.mu.trainable_variables))

        # update target networks

        # updating q_mu network
        
        temp1 = np.array(self.q_mu_target.get_weights())
        temp2 = np.array(self.q_mu.get_weights())
        temp3 = self.tau*temp2 + (1-self.tau)*temp1
        self.q_mu_target.set_weights(temp3)
      

      # updating mu network
        temp1 = np.array(self.mu_target.get_weights())
        temp2 = np.array(self.mu.get_weights())
        temp3 = self.tau*temp2 + (1-self.tau)*temp1
        self.mu_target.set_weights(temp3)
        
    def load(self):
        self.mu.load_weights("mu_target_weights")
        self.q_mu.load_weights("q_mu_target_weights")     
        
        self.mu_target.load_weights("mu_target_weights")
        self.q_mu_target.load_weights("q_mu_target_weights")

    def save(self):
        self.mu.save_weights("mu_weights")
        self.q_mu.save_weights("q_mu_target_weights")     
        
        self.mu_target.save_weights("mu_target_weights")
        self.q_mu_target.save_weights("q_mu_target_weights")

# Main

In [18]:
import gym

In [19]:
env = gym.make("Pendulum-v1")

max_episodes = 50
max_steps = 500
batch_size = 32

gamma = 0.99
tau = 1e-2
buffer_maxlen = 100000
critic_lr = 1e-3
actor_lr = 1e-3

agent = DDPGAgent(env, gamma, tau, buffer_maxlen, critic_lr, actor_lr)

In [20]:
agent.load()

In [21]:
def play(games):
    episode_rewards = []

    for episode in range(games):
        state = env.reset()
        episode_reward = 0

        for step in range(max_steps):
            action = agent.get_action(state, 0)
            next_state, reward, done, _ = env.step(action)
            env.render()
            episode_reward += reward

            if done or step == max_steps-1:
                episode_rewards.append(episode_reward)
                print("Game " + str(episode) + ": " + str(episode_reward))
                break

            state = next_state

    env.close()

In [22]:
play(1)

Game 0: -249.884386064341
